In [4]:

import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

happy = ""
top = -1
    
# Define actions and corresponding labels
actions = ['j','u','p','pp']
seq_length = 30

model = load_model('models/model.h5')

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

cap = cv2.VideoCapture(0)

seq = []
action_seq = []

while cap.isOpened():
    ret, img = cap.read()
    img0 = img.copy()
    
    
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    # cv2.putText(img,f'hi', org=[50,50], fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
    

    if result.multi_hand_landmarks is not None:
        
    
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 4))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]
                
            check = joint[5][0] - joint[17][0]
            point = joint[5][0] - joint[0][0]
                    
                    
            comp_tip_1 = joint[[4,8,12,16], 0:1]    
            comp_tip_2 = joint[[8,12,16,20], 0:1]
                                       
            tip_to_tip = comp_tip_1 - comp_tip_2
                               
            tip_to_tip = tip_to_tip / np.linalg.norm(tip_to_tip)                              
                                       
            zero = joint[[0,0,0,0,0], 1:2 ]
            tip = joint[[4,8,12,16,20], 1:2]
                    
            zero_to_tip = zero - tip
            zero_to_tip = zero_to_tip / np.linalg.norm(zero_to_tip)
                    

                    # Compute angles between joints
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    
            v = v2 - v1 # [20, 3]
                    # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

            angle = np.degrees(angle) # Convert radian to degree
      
            d = np.append(v.flatten(), check)
            d = np.append(d, point)
            d = np.append(d,zero_to_tip)
            d = np.append(d,tip_to_tip)
            
            
            #d = np.concatenate([joint.flatten(), d])
            
            
            d = np.concatenate([d, angle])

                    #d = np.concatenate([joint.flatten(), angle_label])
                    #d = np.concatenate([d, check])
                    
                    
            print(d.shape)
            
            seq.append(d)

            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            if len(seq) < seq_length:
                continue
                
         

            input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)
            print(input_data.shape)
            y_pred = model.predict(input_data).squeeze()
            print(y_pred)

            i_pred = int(np.argmax(y_pred))
            conf = y_pred[i_pred]
            
#             cv2.putText(img, 'yes!!', org=[25,50],
#                     fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

            if conf < 0.95:
                continue

            action = actions[i_pred]
            action_seq.append(action)

            if len(action_seq) < 3:
                continue

            this_action = '?'
            if action_seq[-1] == action_seq[-2] == action_seq[-3]: # 정확한 액션이라면
                this_action = action
                
                #org=(int(res.landmark[0].x * img.shape[1])
                
            
            print(this_action)
            
            if cv2.waitKey(1) == ord('a'):
                if this_action == 'space':
                    this_action = " "
                    happy = happy + this_action
                    top +=1
                    
                if this_action =='backspace':
                    happy = happy[:-1]
                    print(happy)
                    
                else:
                    happy = happy + this_action.upper() # upper() 대문자 변환 함수
                    top += 1
                
            cv2.putText(img, f'{this_action.upper()}', org=[25,50],
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
          
            # happy = ''.join(str(_) for _ in happy)
            print(type(happy))
             
            cv2.putText(img , happy , org=[25,450], fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)        
            cv2.imshow('img', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 393ms/step
[2.9806858e-01 8.4256007e-09 9.1303356e-21 7.0193142e-01]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 34ms/step
[4.7681424e-01 1.0036442e-09 1.8002377e-14 5.2318579e-01]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 40ms/step
[9.9999619e-01 4.2928916e-19 8.2255808e-25 3.8557901e-06]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 43ms/step
[9.99897122e-01 1.27216895e-14 1.77622516e-21 1.02911770e-04]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 33ms/step
[9.99995708e-01 3.82596444e-17 1.28185156e-23 4.31268018e-06]
j
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[9.5982397e-01 1.3740871e-08 2.8050189e-20 4.0176041e-02]
j
<class 'str'>
(86,)
(1, 30, 86)

1/1 [==============================] - 0s 25ms/step
[7.9984850e-01 8.8873952e-05 2.7203206e-02 1.7285949e-01]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[6.8208563e-01 1.9413768e-04 6.6637315e-02 2.5108290e-01]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[0.80494106 0.0023242  0.04326648 0.14946821]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 25ms/step
[0.69084406 0.00248229 0.1323301  0.17434354]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/step
[0.6824596  0.00211234 0.2037068  0.11172122]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/step
[0.55916053 0.00170837 0.3706844  0.06844666]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/step
[0.66784066 0.00173007 0.3192259  0.01120346]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[0.52999544 0.0009897  0.43160623 0.03740859]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/s

(86,)
(1, 30, 86)
1/1 [==============================] - 0s 17ms/step
[0.18124793 0.01520621 0.42145288 0.38209298]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 18ms/step
[0.16329178 0.01287153 0.49701887 0.32681775]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 25ms/step
[0.08113533 0.11372736 0.28317496 0.5219624 ]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 27ms/step
[0.07814512 0.056343   0.38388997 0.4816219 ]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 32ms/step
[0.01017142 0.5513586  0.03493712 0.40353286]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 32ms/step
[0.05188902 0.7643922  0.16270651 0.02101237]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[3.7380990e-03 3.6431840e-01 4.1696612e-07 6.3194317e-01]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/step
[0.09516346 0.8368591  0.00111303 0.06686432]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 16ms/step
[1.8467952e-02 3.7569428e-04 1.3127815e-04 9.8102504e-01]
?
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[2.0612022e-02 1.1332284e-04 9.0034300e-05 9.7918463e-01]
pp
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 25ms/step
[2.0587914e-02 4.1219235e-05 9.2246308e-05 9.7927856e-01]
pp
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[4.5697555e-02 6.1856597e-05 1.5481406e-04 9.5408571e-01]
pp
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 26ms/step
[3.5932560e-02 2.2108461e-05 1.4291955e-04 9.6390247e-01]
pp
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 32ms/step
[3.0194465e-02 4.8286223e-05 2.1091380e-04 9.6954632e-01]
pp
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/step
[4.9547452e-02 9.2796850e-05 2.1833493e-04 9.5014149e-01]
pp
<class 'str'>
(86,

1/1 [==============================] - 0s 24ms/step
[2.2327627e-05 1.2283014e-29 1.0446549e-07 9.9997759e-01]
?
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 19ms/step
[9.9711454e-01 8.3689463e-13 2.8854983e-03 1.8944512e-08]
?
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 17ms/step
[9.9982291e-01 1.5199562e-13 1.7707441e-04 9.2185672e-09]
?
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 32ms/step
[9.999206e-01 7.365141e-15 7.942200e-05 1.077943e-08]
j
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 27ms/step
[4.2893626e-05 5.3353784e-31 6.6757971e-01 3.3237743e-01]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[9.5127875e-01 1.2781266e-11 4.8721146e-02 8.2839520e-08]
j
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 25ms/step
[9.9973613e-01 1.8428920e-13 2.6389572e-04 2.2298433e-10]
j
<class 'str'>
(86,)
(1, 30, 86)
1/1 [=======

...

[9.3670166e-01 6.2032886e-02 7.2139641e-14 1.2655027e-03]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/step
[5.5714083e-22 1.0000000e+00 1.2434926e-34 4.3506151e-31]
u
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 32ms/step
[4.2470437e-17 1.0000000e+00 5.5186145e-36 1.6828636e-29]
u
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/step
[1.1501552e-14 1.0000000e+00 2.8973166e-31 2.8130174e-19]
u
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/step
[2.1799517e-11 1.0000000e+00 1.4373712e-30 3.1672141e-20]
u
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[1.1747908e-09 1.0000000e+00 2.2306037e-31 6.0358366e-22]
u
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 28ms/step
[2.3349676e-06 9.9999762e-01 0.0000000e+00 2.4006498e-19]
u
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[7.23942

1/1 [==============================] - 0s 17ms/step
[8.7915152e-01 1.0136038e-17 1.2084846e-01 4.9718778e-11]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 25ms/step
[7.8620648e-01 2.7864135e-19 2.1379349e-01 1.1034054e-09]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 27ms/step
[7.4363303e-01 7.0995250e-04 2.5490117e-01 7.5579819e-04]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 18ms/step
[0.2741669  0.6429571  0.0760983  0.00677773]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 18ms/step
[6.2236068e-04 9.7337900e-18 9.9937516e-01 2.5607749e-06]
p
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/step
[1.0421711e-01 3.3235585e-04 8.9519280e-01 2.5780802e-04]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/step
[9.9999249e-01 7.6788285e-16 7.4560407e-06 1.0764839e-15]
?
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 21ms/step
[1.2189302e-03 6.3454259e-10 9.9

1/1 [==============================] - 0s 16ms/step
[6.0706767e-03 8.8547735e-04 9.6378851e-01 2.9255275e-02]
?
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/step
[9.2175525e-01 9.9449851e-11 7.8244731e-02 3.9026078e-12]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 32ms/step
[3.0526176e-02 7.0522576e-20 9.6947384e-01 2.9929101e-09]
?
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/step
[4.9452248e-01 7.0186368e-23 5.0546193e-01 1.5525929e-05]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 17ms/step
[8.8444978e-02 1.8199695e-12 9.1155505e-01 4.7124895e-09]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[8.5254377e-01 8.3176025e-22 1.4741954e-01 3.6638958e-05]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/step
[1.4068690e-03 1.8300311e-10 9.9858046e-01 1.2720308e-05]
p
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[1.9

<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[2.8981499e-03 9.1493440e-01 2.4103521e-11 8.2167394e-02]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 17ms/step
[1.0646491e-03 9.9663711e-01 7.2644421e-12 2.2982352e-03]
?
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[5.749135e-04 9.993893e-01 5.159559e-12 3.577495e-05]
u
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 25ms/step
[1.4930742e-04 9.9984312e-01 1.4769280e-12 7.4632781e-06]
u
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 16ms/step
[7.5274518e-05 9.9992335e-01 2.2872446e-12 1.4513265e-06]
u
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 25ms/step
[9.8816961e-02 9.0105098e-01 1.8968045e-07 1.3186577e-04]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 17ms/step
[0.8463686  0.01737535 0.10703453 0.02922149]
(86,)
(1, 30, 86)
1/1 [===================

1/1 [==============================] - 0s 32ms/step
[2.5423772e-15 0.0000000e+00 9.9880087e-01 1.1990645e-03]
p
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 17ms/step
[9.0814002e-16 0.0000000e+00 9.9980432e-01 1.9565884e-04]
p
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[1.4138302e-15 0.0000000e+00 9.9955434e-01 4.4558989e-04]
p
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 33ms/step
[1.2376836e-14 0.0000000e+00 9.9852437e-01 1.4757128e-03]
p
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 24ms/step
[1.07713304e-13 0.00000000e+00 9.98409748e-01 1.59025355e-03]
p
<class 'str'>
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 17ms/step
[5.54604323e-13 6.11448076e-30 1.17888026e-01 8.82111967e-01]
(86,)
(1, 30, 86)
1/1 [==============================] - 0s 25ms/step
[3.3282319e-03 9.2371032e-08 4.5112504e-08 9.9667156e-01]
?
<class 'str'>
(86,)
(1, 30, 86)
